<a href="https://colab.research.google.com/github/Pomaccel/DADS5001-Data-Analytics-and-Data-Science-Tools-and-Programming/blob/main/ml_6610422013_AS02_Predict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [81]:
#install package
!pip install pandas
!pip install numpy
!pip install requests
!pip install lxml
!pip install yfinance
!pip install cython

In [ ]:
!wget http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz
!tar -xzvf ta-lib-0.4.0-src.tar.gz
%cd ta-lib
!./configure --prefix=/usr
!make
!make install


--2024-02-24 14:47:03--  http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz
Resolving prdownloads.sourceforge.net (prdownloads.sourceforge.net)... 204.68.111.105
Connecting to prdownloads.sourceforge.net (prdownloads.sourceforge.net)|204.68.111.105|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.sourceforge.net/project/ta-lib/ta-lib/0.4.0/ta-lib-0.4.0-src.tar.gz [following]
--2024-02-24 14:47:03--  http://downloads.sourceforge.net/project/ta-lib/ta-lib/0.4.0/ta-lib-0.4.0-src.tar.gz
Resolving downloads.sourceforge.net (downloads.sourceforge.net)... 204.68.111.105
Reusing existing connection to prdownloads.sourceforge.net:80.
HTTP request sent, awaiting response... 302 Found
Location: http://cfhcable.dl.sourceforge.net/project/ta-lib/ta-lib/0.4.0/ta-lib-0.4.0-src.tar.gz [following]
--2024-02-24 14:47:03--  http://cfhcable.dl.sourceforge.net/project/ta-lib/ta-lib/0.4.0/ta-lib-0.4.0-src.tar.gz
Resolving cfhcable.dl.s

In [ ]:
!pip install TA-lib

In [ ]:

import numpy as np
import pandas as pd
import yfinance as yf

# Technical Indicators
import talib as ta

# Plotting graphs
import matplotlib.pyplot as plt

# Machine learning
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.metrics import classification_report

In [ ]:
xom = yf.Ticker('XOM')
interval = '1h'
start = '2023-1-1'
end = '2024-2-1'
df = xom.history(interval=interval,start=start,end=end)
df

In [ ]:
df = df.loc[:,"Open":"Close"]
df['S_10'] = df['Close'].rolling(window=10).mean()
df['Corr'] = df['Close'].rolling(window=10).corr(df['S_10'])
df['RSI'] = ta.RSI(np.array(df['Close']), timeperiod =10)
df['Open-Close'] = df['Open'] - df['Close'].shift(1)
df['Open-Open'] = df['Open'] - df['Open'].shift(1)
df = df.dropna()
X = df.iloc[:,:9]
xom = df.copy()
X

In [ ]:
y = np.where(df['Close'].shift(-1) > df['Close'],1,-1)
y


In [ ]:



split = int(0.7*len(df))
X_train, X_test, y_train, y_test = X[:split], X[split:], y[:split], y[split:]

In [ ]:
model = LogisticRegression(max_iter = 1000)
model = model.fit (X_train,y_train)

In [ ]:
pd.DataFrame(zip(X.columns, np.transpose(model.coef_)))

In [ ]:
# Specify the date for prediction
prediction_date = '2024-02-21'
probability = model.predict_proba(X_test)
probability

In [ ]:
predicted = model.predict(X_test)
predicted

In [ ]:
print(metrics.classification_report(y_test,predicted))

In [ ]:
print(metrics.confusion_matrix(y_test, predicted))

In [ ]:
print(model.score(X_test,y_test))

In [ ]:
cross_val = cross_val_score(model, X, y, scoring='accuracy', cv=10)
print(cross_val)
print(cross_val.mean())

In [ ]:
df['Predicted_Signal'] = model.predict(X)
df['xom_returns'] = np.log(df['Close']/df['Close'].shift(1))
Cumulative_Nitfy_returns = np.cumsum(df[split:]['xom_returns'])

df['Strategy_returns'] = df['xom_returns']* df['Predicted_Signal'].shift(1)
Cumulative_Strategy_returns = np.cumsum(df[split:]['Strategy_returns'])

plt.figure(figsize=(10,5))
plt.plot(Cumulative_Nitfy_returns, color='r', label = 'xom Returns')
plt.plot(Cumulative_Strategy_returns, color='g', label = 'Strategy Returns')

plt.legend()
plt.show()

In [ ]:
lpg = yf.Ticker('NOG')
interval = '1h'
start = '2023-1-31'
end = '2024-2-1'
lpgdf = lpg.history(interval=interval,start=start,end=end)
lpgdf

In [ ]:
newdf = pd.merge(xom.iloc[:,:4],lpgdf.iloc[:,:4],how = "inner",left_index=True,right_index=True)
newdf

In [ ]:
newdf.columns

In [ ]:
newdf['S_10Y'] = newdf['Close_y'].rolling(window=10).mean()
newdf['CorrY'] = newdf['Close_y'].rolling(window=10).corr(newdf['S_10Y'])
# new indicator
fastk, fastd = ta.STOCHRSI(newdf['Close_y'], timeperiod=14, fastk_period=5, fastd_period=3, fastd_matype=0)
MOM = ta.MOM(newdf['Close_y'], timeperiod=10)
macd, macdsignal, macdhist = ta.MACD(newdf['Close_y'], fastperiod=12, slowperiod=26, signalperiod=9)
adxr = ta.ADXR(newdf['High_y'], newdf['Low_y'], newdf['Close_y'], timeperiod=14)

newdf['StochRSI_K'] = fastk
newdf['StochRSI_D'] = fastd
newdf['Momentum'] = MOM
newdf['Macd'] = macd
newdf['macdsignal'] = macdsignal
newdf['macdhist'] = macdhist
newdf['ADXR'] = macdhist

newdf['OpenX-CloseX'] = newdf['Open_x'] - newdf['Close_x'].shift(1)
newdf['OpenX-OpenX'] = newdf['Open_x'] - newdf['Open_x'].shift(1)
newdf = newdf.dropna()
combinenew = newdf
combinenew

In [ ]:
combinenew['ynew']= np.where(newdf['Close_x'].shift(-1) > newdf['Close_x'],1,-1)
Xnew = combinenew.iloc[:,:-1]
ynew = combinenew.iloc[:,-1:]

In [ ]:
'''split = int(0.7*len(df))
xnew_train, xnew_test, ynew_train, ynew_test = Xnew[:split], Xnew[split:], ynew[:split], ynew[split:]'''


trainnew, testnew = train_test_split(combinenew,test_size=0.3,random_state = 0,stratify = combinenew['ynew'])
xnew_train = trainnew[trainnew.columns[:-1]]
ynew_train = trainnew[trainnew.columns[-1:]]
xnew_test = testnew[testnew.columns[:-1]]
ynew_test = testnew[testnew.columns[-1:]]

In [ ]:
modelnew = LogisticRegression(max_iter = 1000)
modelnew = modelnew.fit (xnew_train,ynew_train)

In [ ]:
pd.DataFrame(zip(newdf.columns, np.transpose(modelnew.coef_)))

In [ ]:
probability_new = modelnew.predict_proba(xnew_test)
probability_new

In [ ]:
predicted_new = modelnew.predict(xnew_test)
predicted_new

In [ ]:
print(metrics.classification_report(ynew_test,predicted_new))

In [ ]:
print(metrics.confusion_matrix(ynew_test, predicted_new))

In [ ]:
print(modelnew.score(xnew_test,ynew_test))

In [ ]:
cross_val_new = cross_val_score(modelnew, Xnew, ynew, scoring='accuracy', cv=10)
print(cross_val_new)
print(cross_val_new.mean())

In [ ]:
df['Predicted_Signal'] = model.predict(X)
df['xom_returns'] = np.log(df['Close']/df['Close'].shift(1))
Cumulative_Nitfy_returns = np.cumsum(df[split:]['xom_returns'])
df['Strategy_returns'] = df['xom_returns']* df['Predicted_Signal'].shift(1)
Cumulative_Strategy_returns = np.cumsum(df[split:]['Strategy_returns'])


newdf['Predicted_Signal_new'] = modelnew.predict(Xnew)
newdf['lpg trend'] = np.log(newdf['Close_y']/newdf['Close_y'].shift(1))
newdf['Strategy_returns2'] = newdf['lpg trend']* newdf['Predicted_Signal_new'].shift(1)
Cumulative_Strategy_returns2 = np.cumsum(newdf[split:]['Strategy_returns2'])



plt.figure(figsize=(10,5))

plt.plot(Cumulative_Nitfy_returns, color='r', label = 'xom Returns')
plt.plot(Cumulative_Strategy_returns, color='g', label = 'Strategy Returns')
plt.plot(Cumulative_Strategy_returns2, color='b', label = 'Strategy Returns2')

plt.legend()
plt.show()